In [1]:
from tensorflow.keras import models, layers
from tensorflow import keras
from data_handler import get_selected_data
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
import numpy as np

2022-06-22 23:52:31.737112: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-22 23:52:31.737138: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = get_selected_data()

In [3]:
trajectories = [tmd["traj_data"] for tmd in data]
classes = [tmd["class"] for tmd in data]

In [4]:
le = preprocessing.LabelEncoder()
le.fit(classes)
for cls, lbl in zip(le.classes_, le.transform(le.classes_)):
    print(lbl, cls)
labeled_classes = le.transform(classes)
print("\nAll classes labeled:", labeled_classes)

0 bike
1 bus
2 car
3 train
4 walk

All classes labeled: [4 2 2 ... 1 4 1]


In [122]:
X_train, X_test, y_train, y_test = train_test_split(
    trajectories, labeled_classes, stratify=labeled_classes, random_state=1, test_size=0.25,
)

X_validation, X_test, y_validation, y_test = train_test_split(
    X_test, y_test, stratify=y_test, random_state=1, test_size=0.15,
)

In [147]:
def prepare_data(trajs, cls, hist_size=75):
    X, y = [], []
    for traj, cls in zip(trajs, cls):
        traj_len = traj.shape[0]
        temp_traj = np.array([traj[:,1], traj[:, 0], traj[:, 2]]).T
        assert temp_traj.shape == traj.shape, f"{temp_traj.shape} != {traj.shape}"
        for i in range(0, min(traj_len - hist_size, 100), 20):
            t1 = traj[i:i+hist_size]
            _y = [0] * le.classes_.shape[0]
            _y[cls] = 1
            
            X.append(t1)
            y.append(cls)
    return np.array(X), np.array(y)

In [148]:
print(len(X_train))

2957


In [149]:
last_n_points = 100
tX, ty = prepare_data(X_train, y_train, last_n_points)
vX, vy = prepare_data(X_validation, y_validation, last_n_points)

In [150]:
model = keras.Sequential()


# layers
model.add(layers.LSTM(30, input_shape=(last_n_points, 3), return_sequences=True))
model.add(layers.LSTM(10, input_shape=(last_n_points, 3)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


es = keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
mc = keras.callbacks.ModelCheckpoint('best_model_3.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)

model.fit(tX, ty, epochs=100, callbacks=[mc, es])

Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_81 (LSTM)              (None, 100, 30)           4080      
                                                                 
 lstm_82 (LSTM)              (None, 10)                1640      
                                                                 
 dense_142 (Dense)           (None, 30)                330       
                                                                 
 dense_143 (Dense)           (None, 5)                 155       
                                                                 
Total params: 6,205
Trainable params: 6,205
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
430/431 [============================>.] - ETA: 0s - loss: 1.4686 - accuracy: 0.3153
Epoch 1: accuracy improved from -inf to 0.31532, saving model to best_model_3.h5
431/4

431/431 [==============================] - 10s 22ms/step - loss: 1.3883 - accuracy: 0.3712
Epoch 30/100
430/431 [============================>.] - ETA: 0s - loss: 1.3937 - accuracy: 0.3661
Epoch 30: accuracy did not improve from 0.37117
431/431 [==============================] - 9s 22ms/step - loss: 1.3935 - accuracy: 0.3663
Epoch 31/100
430/431 [============================>.] - ETA: 0s - loss: 1.3884 - accuracy: 0.3695
Epoch 31: accuracy did not improve from 0.37117
431/431 [==============================] - 9s 22ms/step - loss: 1.3881 - accuracy: 0.3695
Epoch 32/100
429/431 [============================>.] - ETA: 0s - loss: 1.3896 - accuracy: 0.3657
Epoch 32: accuracy did not improve from 0.37117
431/431 [==============================] - 9s 22ms/step - loss: 1.3902 - accuracy: 0.3654
Epoch 33/100
430/431 [============================>.] - ETA: 0s - loss: 1.3884 - accuracy: 0.3679
Epoch 33: accuracy did not improve from 0.37117
431/431 [==============================] - 9s 22ms/step

In [64]:
def predict(trajs, model, hist_size=75, cls = None):
    if cls is None:
        cls = [""] * len(trajs)
    for traj, cl in zip(trajs, cls):
        X, _ = prepare_data([traj], [0], hist_size)
        pred = model.predict(X)
        print(pred[-1], np.argmax(pred[-1], axis=0), cl)

In [152]:
predict(X_validation[:10], model, last_n_points, y_validation[:10])

1/1 [==============================] - 0s 385ms/step
[0.14811525 0.32233605 0.14321806 0.0505119  0.33581874] 4 1
1/1 [==============================] - 0s 12ms/step
[0.14363262 0.38522276 0.11807641 0.07436059 0.27870762] 1 2
1/1 [==============================] - 0s 12ms/step
[0.1558035  0.31914786 0.13923354 0.04800889 0.33780625] 4 4
1/1 [==============================] - 0s 13ms/step
[0.33415422 0.28711057 0.0669546  0.01799346 0.2937872 ] 0 1
1/1 [==============================] - 0s 12ms/step
[0.38513163 0.2629567  0.05506611 0.01436274 0.28248283] 0 1
1/1 [==============================] - 0s 12ms/step
[0.1428393  0.38351902 0.1199802  0.07324211 0.28041935] 1 0
1/1 [==============================] - 0s 13ms/step
[0.34848303 0.28174552 0.06290912 0.01661595 0.29024637] 0 0
1/1 [==============================] - 0s 12ms/step
[0.3852494  0.26292717 0.0550652  0.01434931 0.28240892] 0 0
1/1 [==============================] - 0s 14ms/step
[0.34587815 0.10565725 0.09494802 0.2530000